# Guided Project 2

**Learning Objective:**

* Learn how to adapt the tfx template to an existing model.

In this guided project, we will use the `tfx template` tool to create a TFX pipeline around the covertype dataset.
The goal is to adapt the template pipeline to make use of the model code for the covertype dataset we  already developed in
the first part of this course. 

In [8]:
import os

## Step 1. Environment setup

### Envirnonment Variables

Setup the your Kubeflow pipelines endopoint below the same way you did in guided project 1.

In [1]:
ENDPOINT = 'https://164002aec59f7c0d-dot-us-central1.pipelines.googleusercontent.com/' # Enter your ENDPOINT here.

In [2]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [3]:
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}

env: GOOGLE_CLOUD_PROJECT=qwiklabs-gcp-04-0ad772141888


In [4]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE = 'gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'
CUSTOM_TFX_IMAGE

'gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline'

### `skaffold` tool setup

In [5]:
%%bash

LOCAL_BIN="/home/jupyter/.local/bin"
SKAFFOLD_URI="https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64"

test -d $LOCAL_BIN || mkdir -p $LOCAL_BIN

which skaffold || (
    curl -Lo skaffold $SKAFFOLD_URI &&
    chmod +x skaffold               &&
    mv skaffold $LOCAL_BIN
)

/usr/local/bin/skaffold


Modify the `PATH` environment variable so that `skaffold` is available:

At this point, you shoud see the `skaffold` tool with the command `which`:

In [6]:
!which skaffold

/usr/local/bin/skaffold


## Step 2. Copy the predefined template to your project directory.

In this step, we will create a working pipeline project directory and 
files by copying additional files from a predefined template.

You may give your pipeline a different name by changing the PIPELINE_NAME below. 

This will also become the name of the project directory where your files will be put.

In [9]:
PIPELINE_NAME = "guided_project_2"
PROJECT_DIR = os.path.join(os.path.expanduser("."), PIPELINE_NAME)
PROJECT_DIR

'./guided_project_2'

TFX includes the taxi template with the TFX python package. 

If you are planning to solve a point-wise prediction problem,
including classification and regresssion, this template could be used as a starting point.

The `tfx template copy` CLI command copies predefined template files into your project directory.

In [10]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

2021-11-12 01:48:39.693926: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 01:48:39.693974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Copying taxi pipeline template
model_analysis.ipynb -> ./guided_project_2/model_analysis.ipynb
pipeline.py -> ./guided_project_2/pipeline/pipeline.py
__init__.py -> ./guided_project_2/pipeline/__init__.py
configs.py -> ./guided_project_2/pipeline/configs.py
kubeflow_dag_runner.py -> ./guided_project_2/kubeflow_dag_runner.py
data_validation.ipynb -> ./guided_project_2/data_validation.ipynb
model.py -> ./guided_project_2/models/keras/model.py
constants.py -> ./guided_project_2/models/keras/constants.py
__init

In [11]:
%cd {PROJECT_DIR}

/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/guided_projects/guided_project_2


### Step 3. Browse your copied source files

The TFX template provides basic scaffold files to build a pipeline, including Python source code,
sample data, and Jupyter Notebooks to analyse the output of the pipeline. 

The `taxi` template uses the same Chicago Taxi dataset and ML model as 
the [Airflow Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

Here is brief introduction to each of the Python files:

`pipeline` - This directory contains the definition of the pipeline
* `configs.py` — defines common constants for pipeline runners
* `pipeline.py` — defines TFX components and a pipeline

`models` - This directory contains ML model definitions.
* `features.py`, `features_test.py` — defines features for the model
* `preprocessing.py`, `preprocessing_test.py` — defines preprocessing jobs using tf::Transform

`models/estimator` - This directory contains an Estimator based model.
* `constants.py` — defines constants of the model
* `model.py`, `model_test.py` — defines DNN model using TF estimator

`models/keras` - This directory contains a Keras based model.
* `constants.py` — defines constants of the model
* `model.py`, `model_test.py` — defines DNN model using Keras

`beam_dag_runner.py`, `kubeflow_dag_runner.py` — define runners for each orchestration engine


**Running the tests:**
You might notice that there are some files with `_test.py` in their name. 
These are unit tests of the pipeline and it is recommended to add more unit 
tests as you implement your own pipelines. 
You can run unit tests by supplying the module name of test files with `-m` flag. 
You can usually get a module name by deleting `.py` extension and replacing `/` with `..`

For example:

In [50]:
# !python -m models.features_test
!python -m models.keras.model_test

2021-11-12 03:58:31.844963: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 03:58:31.845007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Running tests under Python 3.7.10: /opt/conda/bin/python
[ RUN      ] ModelTest.testBuildKerasModel
INFO:tensorflow:time(__main__.ModelTest.testBuildKerasModel): 0.0s
I1112 03:58:35.159034 140376418719552 test_util.py:1973] time(__main__.ModelTest.testBuildKerasModel): 0.0s
[  FAILED  ] ModelTest.testBuildKerasModel
[ RUN      ] ModelTest.test_session
[  SKIPPED ] ModelTest.test_session
ERROR: testBuildKerasModel (__main__.ModelTest)
ModelTest.testBuildKerasModel
------------------------------------------------

## Step 4. Create the artifact store bucket

**Note:** You probably already have completed this step in guided project 1, so you may
may skip it if this is the case.

Components in the TFX pipeline will generate outputs for each run as
[ML Metadata Artifacts](https://www.tensorflow.org/tfx/guide/mlmd), and they need to be stored somewhere.
You can use any storage which the KFP cluster can access, and for this example we
will use Google Cloud Storage (GCS).

Let us create this bucket if you haven't created it in guided project 1.
Its name will be `<YOUR_PROJECT>-kubeflowpipelines-default`.

In [13]:
GCS_BUCKET_NAME = GOOGLE_CLOUD_PROJECT + '-kubeflowpipelines-default'
GCS_BUCKET_NAME

'qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default'

In [14]:
!gsutil ls gs://{GCS_BUCKET_NAME} | grep {GCS_BUCKET_NAME} || gsutil mb gs://{GCS_BUCKET_NAME}

gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/data/
gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/jobs/
gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/staging/
gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/tfx-template/
gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/tfx_pipeline_output/
gs://qwiklabs-gcp-04-0ad772141888-kubeflowpipelines-default/tmp/


## Step 5. Change the dataset

Now we need to have the TFX pipeline dataset point to the GCS bucket
where our covertype data is located. For that

1. open `kubeflow_dag_runner.py`

1. Set the variable `DATA_PATH` to `gs://workshop-datasets/covertype/small` which contains the covertype dataset


## Step 6. Change the pre-processing module

At this step we want to reuse the `features.py` and `preprocessing.py` we already have for the covertype dataset. To do that, we will just copy these files over the template ones:

In [20]:
FEATURE_PY  = '../../tfx_pipelines/pipeline/solutions/pipeline/features.py'
PREPROC_PY = '../../tfx_pipelines/pipeline/solutions/pipeline/preprocessing.py'

In [21]:
!cp {FEATURE_PY} ./models/features.py
!cp {PREPROC_PY} ./models/preprocessing.py

cp: cannot stat '../../tfx_pipelines/pipeline/solutions/pipeline/features.py': No such file or directory
cp: cannot stat '../../tfx_pipelines/pipeline/solutions/pipeline/preprocessing.py': No such file or directory


Now when you run the tests in the two cells below they should fail 
because they were written for the template taxi dataset. 

**Exercise:** Modify the tests `features_test.py` and `preprocessing_test.py` as well as possibly the original modules until the tests pass.

In [27]:
!python -m models.features_test

2021-11-12 02:14:33.245553: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 02:14:33.245622: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Running tests under Python 3.7.10: /opt/conda/bin/python
[ RUN      ] FeaturesTest.testNumberFeatures
INFO:tensorflow:time(__main__.FeaturesTest.testNumberFeatures): 0.0s
I1112 02:14:36.160474 139688828987200 test_util.py:1973] time(__main__.FeaturesTest.testNumberFeatures): 0.0s
[       OK ] FeaturesTest.testNumberFeatures
[ RUN      ] FeaturesTest.testTransformedName
INFO:tensorflow:time(__main__.FeaturesTest.testTransformedName): 0.0s
I1112 02:14:36.160954 139688828987200 test_util.py:1973] time(__main__.Fea

In [38]:
!python -m models.preprocessing_test

2021-11-12 02:24:24.907804: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 02:24:24.907856: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Running tests under Python 3.7.10: /opt/conda/bin/python
[ RUN      ] PreprocessingTest.testPreprocessingFn
INFO:tensorflow:time(__main__.PreprocessingTest.testPreprocessingFn): 0.0s
I1112 02:24:27.824175 140311220856640 test_util.py:1973] time(__main__.PreprocessingTest.testPreprocessingFn): 0.0s
[       OK ] PreprocessingTest.testPreprocessingFn
[ RUN      ] PreprocessingTest.test_session
[  SKIPPED ] PreprocessingTest.test_session
----------------------------------------------------------------------
Ran 2 t

## Step 6. Change the model

Similarly as for the pre-processing we want to reuse the model we develop for the covertype dataset,
so we will simply copy it over the template model:

In [ ]:
MODEL_PY  = '../../tfx_pipelines/pipeline/solutions/pipeline/model.py'

In [ ]:
!cp {MODEL_PY} ./models/keras/model.py

**Exercise:** Move the constants defined in `model.py` into `constants.py` and import them from `model.py` to respect the template structure.

## Step 7. Run the covertype TFX pipeline

Let's create a TFX pipeline using the `tfx pipeline create` command.

**Note:** When creating a pipeline for KFP, we need a container image which will 
be used to run our pipeline. And skaffold will build the image for us. Because `skaffold`
pulls base images from the docker hub, it will take 5~10 minutes when we build
the image for the first time, but it will take much less time from the second build.

In [ ]:
!tfx pipeline create  \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

While creating a pipeline, `Dockerfile` and `build.yaml` will be generated to build a Docker image.

Don't forget to add these files to the source control system (for example, git) along with other source files.

A pipeline definition file for [argo](https://argoproj.github.io/argo/) will be generated, too. 
The name of this file is `${PIPELINE_NAME}.tar.gz.` 
For example, it will be `guided_project_2.tar.gz` if the name of your pipeline is `guided_project_1`. 
It is recommended NOT to include this pipeline definition file into source control, because it will be generated from other Python files and will be updated whenever you update the pipeline. For your convenience, this file is already listed in `.gitignore` which is generated automatically.

Now start an execution run with the newly created pipeline using the `tfx run create` command.

**Note:** You may see the following error `Error importing tfx_bsl_extension.coders.` Please ignore it.

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

Or, you can also run the pipeline in the KFP Dashboard. The new execution run will be listed 
under Experiments in the KFP Dashboard. 
Clicking into the experiment will allow you to monitor progress and visualize 
the artifacts created during the execution run.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from 
the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, 
you will be able to find the pipeline, and access a wealth of information about the pipeline. 
For example, you can find your runs under the Experiments menu, and when you open your
execution run under Experiments you can find all your artifacts from the pipeline under Artifacts menu.

**Note:** If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard.

One of the major sources of failure is permission related problems. 
Please make sure your KFP cluster has permissions to access Google Cloud APIs.
This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up),
or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## Step 8. Add components for data validation.

In this step, you will add components for data validation including `StatisticsGen`, `SchemaGen`, and `ExampleValidator`.
If you are interested in data validation, please see 
[Get started with Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

**Double-click to change directory to pipeline and double-click again to open** `pipeline.py`. 
Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline.
(Tip: search for comments containing TODO(step 5):). Make sure to save `pipeline.py` after you edit it.

You now need to update the existing pipeline with modified pipeline definition. Use the `tfx pipeline update` command to update your pipeline, followed by the `tfx run create` command to create a new execution run of your updated pipeline.

In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

In [ ]:
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### Check pipeline outputs

Visit the KFP dashboard to find pipeline outputs in the page for your pipeline run. Click the Experiments tab on the left, and All runs in the Experiments page. You should be able to find the latest run under the name of your pipeline.

See link below to access the dashboard:

In [ ]:
print('https://' + ENDPOINT)

## Step 9. Add components for training

In this step, you will add components for training and model validation including `Transform`, `Trainer`, `ResolverNode`, `Evaluator`, and `Pusher`.

**Double-click to open** `pipeline.py`. Find and uncomment the 5 lines which add `Transform`, `Trainer`, `ResolverNode`, `Evaluator` and `Pusher` to the pipeline. (Tip: search for TODO(step 6):)

**Hints:** 

* In `pipeline.py` make sure you turn the cache of by setting `enable_cache=False` for debugging purposes (otherwise components that have been previously run won't be).

* In `pipeline.py`, you'll need to set `infer_feature_shape=False`, otherwise you'll run into sparse/dense tensor mismatch.

* In `pipeline.py`, you'll need to set the label `big_tipper` from the default template to the right label for our dataset in 

```python
tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='big_tipper')],
```

* In `configs.py`, adapt the values of the variables `TRAIN_NUM_STEPS` and `EVAL_NUM_STEPS` to match what we had in the original covertype pipeline.

As you did before, you now need to update the existing pipeline with the modified pipeline definition. The instructions are the same as Step 5. Update the pipeline using `tfx pipeline update`, and create an execution run using `tfx run create`.

Verify that the pipeline DAG has changed accordingly in the Kubeflow UI:

In [41]:
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

2021-11-12 02:34:18.628151: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 02:34:18.628203: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline -> gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline: Not found. Building
[Skaffold] Starting build...
[Skaffold] Building [gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline]...
[Skaffo

In [42]:
print("https://" + ENDPOINT)

https://https://164002aec59f7c0d-dot-us-central1.pipelines.googleusercontent.com/


In [43]:
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

2021-11-12 02:34:42.255797: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 02:34:42.255843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating a run for pipeline: guided_project_2
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: guided_project_2
+------------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------------+
| pipeline_name    | run_id                               | status   | created_at                | link         

When this execution run finishes successfully, you have now created and run your first TFX pipeline in AI Platform Pipelines!

## Step 10. Try Cloud AI Platform Training and Prediction with KFP

TFX interoperates with several managed GCP services, such as [Cloud AI Platform for Training and Prediction](https://cloud.google.com/ai-platform/). You can set your `Trainer` component to use Cloud AI Platform Training, a managed service for training ML models. Moreover, when your model is built and ready to be served, you can push your model to Cloud AI Platform Prediction for serving. In this step, we will set our `Trainer` and `Pusher` component to use Cloud AI Platform services.

Before editing files, you might first have to enable AI Platform Training & Prediction API.


**Double-click pipeline to change directory, and double-click to open** `configs.py`. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` and `GCP_AI_PLATFORM_SERVING_ARGS`. We will use our custom built container image to train a model in Cloud AI Platform Training, so we should set `masterConfig.imageUri` in `GCP_AI_PLATFORM_TRAINING_ARGS` to the same value as `CUSTOM_TFX_IMAGE` above.

**Change directory one level up, and double-click to open** `kubeflow_dag_runner.py`. Uncomment `ai_platform_training_args` and `ai_platform_serving_args`.

Update the pipeline and create an execution run as we did in step 5 and 6.

In [46]:
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

2021-11-12 03:03:45.033927: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 03:03:45.033976: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline -> gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline: Not found. Building
[Skaffold] Starting build...
[Skaffold] Building [gcr.io/qwiklabs-gcp-04-0ad772141888/tfx-pipeline]...
[Skaffo

In [47]:
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

2021-11-12 03:04:04.666732: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-12 03:04:04.666782: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating a run for pipeline: guided_project_2
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: guided_project_2
+------------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------------+
| pipeline_name    | run_id                               | status   | created_at                | link         

You can find your training jobs in [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). If your pipeline completed successfully, you can find your model in [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## License

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.